In [3]:
import pandas as pd
import requests
import urllib3
from bs4 import BeautifulSoup
from os import listdir
from tqdm import tqdm as progression_bar
import timeit

In [4]:
path = 'C:/Users/Marie/Desktop/Bike Sharing/Full Data/Cleaned Data/'
print(listdir(path))
data_day = pd.read_csv(path + listdir(path)[0])


data_day['date_str'] = data_day['year'].astype(str) +'/'+ \
data_day['month'].apply(lambda x: '0'+str(x) if len(str(x))==1 else str(x))+'/'+ \
data_day['day'].apply(lambda x: '0'+str(x) if len(str(x))==1 else str(x))


dates = pd.unique(data_day.date_str)

['full_daily_data.csv', 'full_hourly_data.csv', 'log.txt', 'stations.csv', 'Weather Data']


In [ ]:
url_base = 'http://www.historique-meteo.net/amerique-du-nord/etats-unis/washington-dc/'

with open(path + 'log_year.txt', 'w') as file:
    file.write('LOG FILE : Scraping Weather Data \n')
    file.write(url_base + '\n\n')

# List of tables
tables = []
year = dates[0][:4]

start = timeit.default_timer()
for index, date in enumerate(dates):
    
    # Open url for the date
    r = requests.get(url_base + date + '/')
    
    # Write status code in the log file
    with open(path + 'log_year.txt', 'a') as file:
        file.write('Date : {} || Status : {}\n'.format(date, r.status_code))
    
    # Checking the status code : 200 is ok
    if r.status_code == 200:
        # Getting the data into the right shape for the building of the dataframe
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find_all("table")[0]
        table = table.contents[3]
        
        # Making the web data into a dataframe
        new_table = pd.DataFrame(columns=range(0,6), index = range(0,14)) 

        row_marker = 0
        for row in table.find_all('tr')[:-1]:
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                new_table.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            row_marker +=1
        # Reshaping the dataframe
        new_table = new_table.iloc[:,[0,3]]   
        new_table.columns = ['field', date]
        
        
    if year != date[:4]:
        df = tables[0]
        try :
            for t in tables[1:]:
                df = df.merge(t, how = 'outer', on = 'field')
                df.to_csv(path + 'Weather Data/weather_data_'+year+'.csv', index = False)
            with open(path + 'log_year.txt', 'a') as file:
                file.write('Managed to write back up file\n')
        except ValueError as e: 
            with open(path + 'log_year.txt', 'a') as file:
                file.write('Did not write back up file !!\n')
                file.write(str(e)+'\n')
        year = date[:4]
        tables = [new_table]
    else : 
        tables.append(new_table)
        
        
file.close()
end = timeit.default_timer()
with open(path + 'log_year.txt', 'a') as file:
    file.write('Execution Time : '+ str(end - start))